In [2]:
import os
print(os.getcwd())  # 현재 작업 디렉토리 확인


c:\Users\User\Desktop\빅프로젝트\BigProject


In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, MultiLineString

# 파일 경로 설정
csv_path = "./testing/광진구_clusters_route 1.csv"
geojson_path = "./testing/road.geojson"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 도로 네트워크 로드
gdf_roads = gpd.read_file(geojson_path)

# 도로 네트워크 그래프 생성
G = nx.Graph()

# GeoJSON에서 도로 데이터 추가 (MultiLineString 포함)
for _, row in gdf_roads.iterrows():
    geom = row.geometry
    if isinstance(geom, MultiLineString):
        for line in geom.geoms:
            coords = list(line.coords)
            for i in range(len(coords) - 1):
                G.add_node(coords[i], pos=coords[i])
                G.add_node(coords[i + 1], pos=coords[i + 1])
                G.add_edge(coords[i], coords[i + 1], weight=line.length)
    elif isinstance(geom, LineString):
        coords = list(geom.coords)
        for i in range(len(coords) - 1):
            G.add_node(coords[i], pos=coords[i])
            G.add_node(coords[i + 1], pos=coords[i + 1])
            G.add_edge(coords[i], coords[i + 1], weight=geom.length)

# 노드 개수 확인
if len(G.nodes) == 0:
    raise ValueError("도로 네트워크가 생성되지 않았습니다. road.geojson 데이터를 확인하세요.")

# 0번 클러스터만 필터링
df_cluster_0 = df[df['cluster'] == 0].sort_values(by='order')

# 클러스터 0의 좌표 리스트 (lat, lon 순서)
coordinates = list(zip(df_cluster_0['latitude'], df_cluster_0['longitude']))

# 도로 위의 가장 가까운 노드 찾기
nodes = []
for lat, lon in coordinates:
    closest_node = min(G.nodes, key=lambda node: (node[0] - lon)**2 + (node[1] - lat)**2)
    nodes.append(closest_node)

# 최적 경로 찾기 (도로 네트워크 기반)
route_edges = []
for i in range(len(nodes) - 1):
    path = nx.shortest_path(G, source=nodes[i], target=nodes[i + 1], weight="weight")
    edges = [(path[j], path[j + 1]) for j in range(len(path) - 1)]
    for edge in edges:
        route_edges.append(LineString([edge[0], edge[1]]))

# 경로 데이터를 GeoDataFrame으로 변환
gdf_route = gpd.GeoDataFrame(geometry=route_edges, crs=gdf_roads.crs)

# GeoJSON 파일로 저장
gdf_route.to_file("cluster_0_route.geojson", driver="GeoJSON")

print("GeoJSON 저장 완료: cluster_0_route.geojson")


In [8]:
import folium
import geopandas as gpd

# 파일 경로 설정
road_geojson = "./visualization/testing/road.geojson"
route_geojson = "./visualization/cluster_0_route.geojson"

# 지도 중심 좌표 설정 (서울 광진구 기준)
center = [37.54, 127.08]

# 지도 생성
m = folium.Map(location=center, zoom_start=15)

# 도로(road) GeoJSON 추가 (굵기 증가)
gdf_roads = gpd.read_file(road_geojson)
folium.GeoJson(gdf_roads, name="Roads", style_function=lambda x: {'color': 'gray', 'weight': 3}).add_to(m)

# 최적 경로 GeoJSON 추가 (투명도 적용)
gdf_route = gpd.read_file(route_geojson)
folium.GeoJson(gdf_route, name="Optimized Route", style_function=lambda x: {'color': 'blue', 'weight': 1, 'opacity': 0.3}).add_to(m)

# 지도 저장
map_path = "./road_route_map.html"
m.save(map_path)

print(f"HTML 지도 저장 완료: {map_path}")

HTML 지도 저장 완료: ./road_route_map.html
